# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def get_metrics(df, field, is_numeric=True):
    if is_numeric:
        return [df[field].max(), df[field].min(), df[field].max() - df[field].min(), df[field].mean(), df[field].mode()[0], df[field].median(), df[field].std(), df[field].var()]
    else:
        return [np.NaN, np.NaN, np.NaN, np.NaN, df[field].round().mode()[0], np.NaN, np.NaN, np.NaN]

# 4: ректальная температура, в норме 37,8 (числовое)
# 5: пульс, в норме 30-40 (числовое)
# 8: периферический пульс (категориальное)
#                 1 = нормальный
#                 2 = увеличено
#                 3 = уменьшено
#                 4 = отсутствует
# 9: слизистые оболочки (категориальное)
#                 1 = нормальный розовый
#                 2 = ярко-розовый
#                 3 = бледно-розовый
#                 4 = бледно-синюшный
#                 5 = ярко-красный / инжектированный
#                 6 = темно-синюшный
# 12: перистальтика (категориальное)
#                 1 = гипермобильный
#                 2 = нормальный
#                 3 = гипомотиль
#                 4 = отсутствует
# 13: вздутие живота (категориальное)
#                 1 = нет
#                 2 = легкий
#                 3 = умеренный
#                 4 = тяжелый
# 19: кол-во эритроцитов в крови, в норме 30-50 (числовое)
# 20: общий белок, в норме 6-7.5 (числовое)

info = (("Ректальная температура", 3, True), ("Пульс", 4, True), ("Периферический пульс", 7, False),
        ("Слизистые оболочки", 8, False), ("Перистальтика", 11, False), ("Вздутие живота", 12, False),
        ("Кол-во эритроцитов", 18, True), ("Общий белок", 19, True))

main = []
horse_data = pd.read_csv("horse_data.csv", header=None, na_values="?")
selected = horse_data.iloc[:, [x[1] for x in info]]
selected.columns=[x[0] for x in info]
for inf in info:
    main.append([inf[0]] + get_metrics(selected, inf[0], inf[2]))
values = pd.DataFrame(main, columns=["Параметр", "Максимум", "Минимум", "Размах", "Среднее", "Мода", "Медиана", "СКО", "Дисперсия"]).set_index("Параметр")
display(HTML(values.to_html()))
display(HTML("<b><u>Выводы:</u></b>"))
print("Ректальная температура: размах небольшой, что характерно для температуры; среднее, мода и медиана говорят, что у большинства лошадей нормальная или незначительно повышенная температура; СКО также не значительно.")
print("Пульс: довольно широкий размах значений, при этом среднее, мода и медиана сильно различаются, а значит, значения распределены неравномерно и у большинства лошадей повышенный пульс.")
print("Периферический пульс: у большинства лошадей нормальный периферический пульс.")
print("Слизистые оболочки: у большинства лошадей нормальные розовые слизистые оболочки.")
print("Перистальтика: у большинства лошадей гипомобильная перистальтика.")
print("Вздутие живота: у большинства лошадей легкое вздутие живота.")
print("Кол-во эритроцитов: среднее значение в пределях нормы и практически совпадает с медианой, это говорит о том, что скорее всего значения рапределены равномерны; СКО также небольшой, хотя размах довольно широкий, т.е. выборка содержит выбросы.")
print("Общий белок: очень большой размах, при том, что небольшие значения моды и медианы; среднее от них отличается довольно сильно, возможно, есть выбросы.")

,Максимум,Минимум,Размах,Среднее,Мода,Медиана,СКО,Дисперсия
Параметр,,,,,,,,
Ректальная температура,40.8,35.4,5.4,38.167917,38.0,38.2,0.732289,0.536247
Пульс,184.0,30.0,154.0,71.913043,48.0,64.0,28.630557,819.708775
Периферический пульс,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Слизистые оболочки,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Перистальтика,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
Вздутие живота,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Кол-во эритроцитов,75.0,23.0,52.0,46.295203,37.0,45.0,10.419335,108.562532
Общий белок,89.0,3.3,85.7,24.456929,6.5,7.5,27.475009,754.876145


Ректальная температура: размах небольшой, что характерно для температуры; среднее, мода и медиана говорят, что у большинства лошадей нормальная или незначительно повышенная температура; СКО также не значительно.
Пульс: довольно широкий размах значений, при этом среднее, мода и медиана сильно различаются, а значит, значения распределены неравномерно и у большинства лошадей повышенный пульс.
Периферический пульс: у большинства лошадей нормальный переферический пульс.
Слизистые оболочки: у большинства лошадей нормальные розовые слизистые оболочки.
Перистальтика: у большинства лошадей гипомобильная перистальтика.
Вздутие живота: у большинства лошадей легкое вздутие живота.
Кол-во эритроцитов: среднее значение в пределях нормы и практически совпадает с медианой, это говорит о том, что скорее всего значения рапределены равномерны; СКО также небольшой, хотя размах довольно широкий, т.е. выборка содержит выбросы.
Общий белок: очень большой размах, при том, что небольшие значения моды и медианы

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [2]:
def get_outliers(df, field):
    q1 = df[field].quantile(0.25)
    q3 = df[field].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)
    outliers = df[df[field].between(df[field].min(), lower_bound, inclusive=True) | df[field].between(upper_bound, df[field].max(), inclusive=True)]
    return outliers[[field]].sort_values(field)
    
display(HTML("<b><u>Выбросы:</u></b>"))
for inf in info:
    if inf[2]:
        res = pd.DataFrame(get_outliers(horse_data, inf[1]))
        res.columns=[inf[0]]
        if len(res):
            display(HTML(res.to_html()))
            
display(HTML("<b><u>Выводы:</u></b>"))
print("Ректальная температура: выбросы сверху и снизу могут быть связаны с болезенным состоянием лошади или погрешностью измерения. Значения выглядят вполне реальными, поэтому необходимости их отсеивать нет.")
print("Пульс: значения пульса 150 и более могут говорить о том, что измерения проводились либо серьезно больной лошади, либо сразу после бега, либо это ошибка измерения. Пульс 184 является слишком высоким и неправдоподобным, поэтому из анализа его следует исключить.")
print("Кол-во эритроцитов: высокие значения концентрации эритроцитов говорят, что скорее всего у лошади обезвоживание. Значения вполне реальные, отсеивать не стоит.")

,Ректальная температура
44,35.4
141,36.0
238,36.1
80,36.4
118,36.5
298,36.5
251,36.6
99,39.6
75,39.7
20,39.9


,Пульс
41,150.0
275,150.0
55,160.0
3,164.0
255,184.0


,Кол-во эритроцитов
35,73.0
49,73.0
4,74.0
144,75.0
151,75.0


Ректальная температура: выбросы сверху и снизу могут быть связаны с болезенным состоянием лошади или погрешностью измерения. Значения выглядят вполне реальными, поэтому необходимости их отсеивать нет.
Пульс: значения пульса 150 и более могут говорить о том, что измерения проводились либо серьезно больной лошади, либо сразу после бега, либо это ошибка измерения. Пульс 184 является слишком высоким и неправдоподобным, поэтому из анализа его следует исключить.
Кол-во эритроцитов: высокие значения концентрации эритроцитов говорят, что скорее всего у лошади обезвоживание. Значения вполне реальные, отсеивать не стоит.


### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [3]:
# не понятно зачем в работе с пропуками надо считать выбросы, видимо, опечатка
# посчитаем кол-во пропусков по каждому столбцу

print("Датафрейм из выбранных столбцов (первые 25)")
display(HTML(selected.iloc[0:25].to_html()))

print("Пропуски")
result = []
rows, cols = selected.shape
for i in range(cols):
    result.append([info[i][0], rows - selected.iloc[:, i].count(), round((rows - selected.iloc[:, i].count()) * 100 / rows, 2)])
nans = pd.DataFrame(result, columns=["Параметр", "Кол-во пропусков", "Процент пропуска"]).set_index("Параметр")
display(HTML(nans.to_html()))

display(HTML("<b><u>Выбор способа замены пропусков:</u></b>"))
print("Больше всего пропусков у параметра \"Периферический пульс\" - 23%. Значение довольно большое, но удалять столбец целиком смысла нет - потеряем много данных.")
print("В целом по всем выбранным столбцам около 10-20% пропущенных данных, поэтому удаляя строки с пустыми значениями также можно потерять множество данных.")
print("Закономерность в появлении пропущенных значений по всем столбцам не прослеживается.")
print("Связи между пульсом и периферическим пульсом не прослеживается: есть как значения уменьшенного периферического пульса при высоком, так и при низком пульсе и наоборот. Поэтому заполнить пропуски руководствуясь другим типом пульса не получится.")
print("Связи между перистальтикой и вздутием живота также не прослеживается: есть как высокая перистальника при отсутствии вздутия, так и с тяжелым и наоборот.")
print("Четкой зависимости между ректальной температуро также нет. Есть лишь некоторая вероятность, что при повышенной температуре будет повышенный пульс. Кроме того, и та и другая - непрерывные величины и восстановить пропущенные значения одной за счет другой также будет довольно проблематично.")
print("\nКак в итоге заполняем пропуски. Для числовых полей без выбросов можно взять среднее арифметическое. Для числовых с выбросами - берем медиану, т.к. она более устойчива к выбросам. Пустые категориальные заполняем модой.")

Датафрейм из выбранных столбцов (первые 25)


,Ректальная температура,Пульс,Периферический пульс,Слизистые оболочки,Перистальтика,Вздутие живота,Кол-во эритроцитов,Общий белок
0,38.5,66.0,3.0,NaN,4.0,4.0,45.0,8.4
1,39.2,88.0,NaN,4.0,4.0,2.0,50.0,85.0
2,38.3,40.0,1.0,3.0,3.0,1.0,33.0,6.7
3,39.1,164.0,1.0,6.0,4.0,4.0,48.0,7.2
4,37.3,104.0,NaN,6.0,NaN,NaN,74.0,7.4
5,NaN,NaN,1.0,3.0,3.0,2.0,NaN,NaN
6,37.9,48.0,1.0,1.0,3.0,3.0,37.0,7.0
7,NaN,60.0,NaN,NaN,4.0,2.0,44.0,8.3
8,NaN,80.0,4.0,3.0,4.0,4.0,38.0,6.2
9,38.3,90.0,NaN,1.0,3.0,1.0,40.0,6.2


Пропуски


,Кол-во пропусков,Процент пропуска
Параметр,,
Ректальная температура,60,20.00
Пульс,24,8.00
Периферический пульс,69,23.00
Слизистые оболочки,47,15.67
Перистальтика,44,14.67
Вздутие живота,56,18.67
Кол-во эритроцитов,29,9.67
Общий белок,33,11.00


Больше всего пропусков у параметра "Периферический пульс" - 23%. Значение довольно большое, но удалять столбец целиком смысла нет - потеряем много данных.
В целом по всем выбранным столбцам около 10-20% пропущенных данных, поэтому удаляя строки с пустыми значениями также можно потерять множество данных.
Закономерность в появлении пропущенных значений по всем столбцам не прослеживается.
Связи между пульсом и периферическим пульсом не прослеживается: есть как значения уменьшенного периферического пульса при высоком, так и при низком пульсе и наоборот. Поэтому заполнить пропуски руководствуясь другим типом пульса не получится.
Связи между перистальтикой и вздутием живота также не прослеживается: есть как высокая перистальника при отсутствии вздутия, так и с тяжелым и наоборот.
Четкой зависимости между ректальной температуро также нет. Есть лишь некоторая вероятность, что при повышенной температуре будет повышенный пульс. Кроме того, и та и другая - непрерывные величины и восстановить проп

In [4]:
def replace_nan(df, field, rtype, vals):
    df[field].fillna(vals.loc[field][rtype], inplace=True)

replaced = pd.DataFrame(selected)
replace_type = ["Медиана", "Медиана", "Мода", "Мода", "Мода", "Мода", "Медиана", "Среднее"]
for i in range(len(info)):
    replace_nan(replaced, info[i][0], replace_type[i], values)
print("Датафрейм без пропусков (первые 25 значений)")
display(HTML(replaced.iloc[0:25].to_html()))

after_replace = []
for inf in info:
    after_replace.append([inf[0]] + get_metrics(replaced, inf[0], inf[2]))
after_replace = pd.DataFrame(after_replace, columns=["Параметр", "Максимум", "Минимум", "Размах", "Среднее", "Мода", "Медиана", "СКО", "Дисперсия"]).set_index("Параметр")
print("Метрики до замены")
display(HTML(values.to_html()))
print("Метрики после замены")
display(HTML(after_replace.to_html()))

print("Как видно из представленных таблиц, после замены пропусков основные метрики изменились для числовых столбцов.")
print("Это неудивительно, т.к. в исходном варианте при подсчете метрик пустые значения были проигнорированы.")

Датафрейм без пропусков (первые 25 значений)


,Ректальная температура,Пульс,Периферический пульс,Слизистые оболочки,Перистальтика,Вздутие живота,Кол-во эритроцитов,Общий белок
0,38.5,66.0,3.0,1.0,4.0,4.0,45.0,8.400000
1,39.2,88.0,1.0,4.0,4.0,2.0,50.0,85.000000
2,38.3,40.0,1.0,3.0,3.0,1.0,33.0,6.700000
3,39.1,164.0,1.0,6.0,4.0,4.0,48.0,7.200000
4,37.3,104.0,1.0,6.0,3.0,1.0,74.0,7.400000
5,38.2,64.0,1.0,3.0,3.0,2.0,45.0,24.456929
6,37.9,48.0,1.0,1.0,3.0,3.0,37.0,7.000000
7,38.2,60.0,1.0,1.0,4.0,2.0,44.0,8.300000
8,38.2,80.0,4.0,3.0,4.0,4.0,38.0,6.200000
9,38.3,90.0,1.0,1.0,3.0,1.0,40.0,6.200000


Метрики до замены


,Максимум,Минимум,Размах,Среднее,Мода,Медиана,СКО,Дисперсия
Параметр,,,,,,,,
Ректальная температура,40.8,35.4,5.4,38.167917,38.0,38.2,0.732289,0.536247
Пульс,184.0,30.0,154.0,71.913043,48.0,64.0,28.630557,819.708775
Периферический пульс,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Слизистые оболочки,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Перистальтика,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
Вздутие живота,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Кол-во эритроцитов,75.0,23.0,52.0,46.295203,37.0,45.0,10.419335,108.562532
Общий белок,89.0,3.3,85.7,24.456929,6.5,7.5,27.475009,754.876145


Метрики после замены


,Максимум,Минимум,Размах,Среднее,Мода,Медиана,СКО,Дисперсия
Параметр,,,,,,,,
Ректальная температура,40.8,35.4,5.4,38.174333,38.200000,38.2,0.654831,0.428804
Пульс,184.0,30.0,154.0,71.280000,64.000000,64.0,27.541545,758.536722
Периферический пульс,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
Слизистые оболочки,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
Перистальтика,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN
Вздутие живота,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
Кол-во эритроцитов,75.0,23.0,52.0,46.170000,45.000000,45.0,9.908584,98.180033
Общий белок,89.0,3.3,85.7,24.456929,24.456929,7.7,25.914514,671.562056


Как видно из представленных таблиц, после замены пропусков основные метрики изменились для числовых столбцов.
Это неудивительно, т.к. в исходном варианте при подсчете метрик пустые значения были проигнорированы.


## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.